In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from random import randint
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import itertools
from joblib import dump, load
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier  


In [ ]:
NomeBase = "baseEngenhariaNumerica4"
NomeBase = 'GLCMMaior'
#NomeBase = "sexoIdade"
#NomeBase = "SIR"
#NomeBase = "siriro"

valor = 1
base = pd.read_csv('../Datasets/' + str(NomeBase)+'.txt')


entradas = base.iloc[:, 0:len(base.columns) -valor].values 

classes = base.iloc[:,len(base.columns)-valor].values

NumColunas = len(base.columns) - valor
caminho = '../resultados/KNN/PesosKNN_Base'+str(NomeBase)
print(classes)

In [ ]:
for i in range (len(base.columns)-valor):
    entradas[:,i] = (entradas[:,i] - np.mean(entradas[:,i]))/entradas[:,i].std(axis = 0)

In [ ]:
def dividir_trei_test(entradas, classes, percentual):
    result = []
    while (len(result)) != len(entradas):
        r = randint(0, len(entradas)-1)
        if r not in result:
            result.append(r)    

    repeticoes = np.zeros(np.max(classes)+1)
    
    for i in range(len(entradas)):
        repeticoes[classes[i]] = repeticoes[classes[i]] + 1
    repeticoes = np.array(repeticoes)
    minimo = int(np.min(repeticoes) * (1 - percentual))
    
    treino = int((minimo) * (np.max(classes)+1) )
    teste = int(len(entradas) - treino)

    entradas_trei = np.zeros((treino, NumColunas))
    entradas_test = np.zeros((teste, NumColunas))
    classes_trei = np.zeros((treino)) 
    classes_test = np.zeros((teste))

 
    contTreino = 0
    contTest = 0
    vetorMinimo = np.zeros(np.max(classes)+1)

    for i in range(len(classes)):    
        if (vetorMinimo[classes[result[i]]] < minimo):            
            entradas_trei[contTreino] = entradas[result[i]]
            classes_trei[contTreino] = classes[result[i]]
   
            contTreino = contTreino + 1
            vetorMinimo[classes[result[i]]] = vetorMinimo[classes[result[i]]] + 1
            
        else:   
            entradas_test[contTest] = entradas[result[i]]            
            classes_test[contTest] = classes[result[i]]
           
            contTest = contTest + 1        


    return entradas_trei, entradas_test, classes_trei, classes_test

In [ ]:
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(classes)
NumClasses = np.max(classes) + 2

entradas_trei, entradas_test, classes_t, classes_te = dividir_trei_test(entradas, classes, 0.2)
contador = np.zeros((3))

classes_test =  np.array(pd.get_dummies(classes_te))
classes_trei =  np.array(pd.get_dummies(classes_t))

In [ ]:
# Modelo sequencial do keras

model = Sequential()

# Camada de entrada
NeuroniosEntrada = 16
model.add(Dense(NeuroniosEntrada, input_dim=NumColunas, activation='relu'))


# Camadas ocultadas
camadasOcultas = 2
NeuroniosOculta = 20

for i in range (camadasOcultas):
    model.add(Dense(NeuroniosOculta, activation='relu'))
    
# Camada de Saída    
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(entradas_trei, classes_t, validation_data=(entradas_test,classes_te), epochs=100, batch_size=10)


In [ ]:
_, accuracy = model.evaluate(entradas, classes)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
print(model.summary())

In [ ]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):   

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Saída correta')
    plt.xlabel('Saída encontrada')    

In [ ]:
y_treino = model.predict(entradas_trei)
y_treino = (y_treino > 0.5)
cm = confusion_matrix(classes_t, y_treino)

plot_confusion_matrix(cm, 
                      classes=['0', '1','FR'],
                      title='Treinamento')
plt.show()

print((cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1]))


In [ ]:
y_teste = model.predict(entradas_test)
y_teste = (y_teste > 0.5)
cmT = confusion_matrix(classes_te, y_teste)
print(cmT)
print((cmT[0][0]+cmT[1][1])/(cmT[0][0]+cmT[0][1]+cmT[1][0]+cmT[1][1]))


plot_confusion_matrix(cmT, 
                      classes=['0', '1', 'FR'],
                      title='Teste')
plt.show()